In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import corner
import astropy.cosmology as cosmo
from astropy.cosmology import Planck18
import astropy.units as u
import copy
from sklearn.mixture import GaussianMixture
from scipy.stats import truncnorm
import scipy.stats as ss

from scipy.interpolate import RectBivariateSpline, interp1d

seed = 1023123283

# Data Generation

In [2]:
variant = 'N1e7_Fiducial_AllDCO_AIS'
with h5py.File("../gmm.h5",'r') as file:
    nc = np.array(file['nc_'+variant+'_all'])
    mean = np.array(file['mean_'+variant+'_all'])
    cov = np.array(file['cov_'+variant+'_all'])
    weight = np.array(file['weight_'+variant+'_all'])
file.close()

In [3]:
gm = GaussianMixture(int(nc))
gm.weights_ = weight
gm.means_ = mean
gm.covariances_ = cov

In [4]:
with h5py.File('../optimal_snr_CosmicExplorerWidebandP1600143.h5', 'r') as inp:
    ms = np.array(inp['ms'])
    osnrs = np.array(inp['SNR'])

osnr_interp = RectBivariateSpline(ms, ms, osnrs)

def optimal_snr(Mcz, logq, dl):
    m1z = Mcz*(np.exp(logq)**(-3/5))*(1+np.exp(logq))**(1/5)
    m2z = Mcz*(np.exp(logq)**(2/5))*(1+np.exp(logq))**(1/5)
    return osnr_interp.ev(m1z, m2z)/dl

def rho(optimal_snr, Theta):
    return optimal_snr*Theta

In [5]:
uncert = {
    'threshold_snr': 8,
    'Theta': 0.05,
    'mcz': 0.03,
    'logq': 0.64
}
rho_th = 8.0

In [6]:
def mcz_add_err(Mcz, rho_obs, uncert, Nsamp):
    Nobs = Mcz.shape[0]
    sigma_Mcz = uncert['threshold_snr']/rho_obs*uncert['mcz']
    Mczo = Mcz+ sigma_Mcz*np.random.randn(Nobs)
    Mczs = np.random.normal(Mczo[:,None], sigma_Mcz[:,None], size=(Nobs,Nsamp))
    return Mczs

def logq_add_err(logq, rho_obs, uncert, Nsamp):
    Nobs = logq.shape[0]
    sigma_logq = uncert['threshold_snr']/rho_obs*uncert['logq']
    bo = -logq/sigma_logq
    logqo = truncnorm.rvs(a=-np.inf*np.ones(Nobs), b=bo, loc=logq, scale=sigma_logq, size=Nobs)

    bs = []
    for i in range(Nobs):
        bs.append(np.repeat(-logqo[i]/sigma_logq[i], Nsamp))
    bs = np.array(bs)
    logqs = truncnorm.rvs(a=-np.inf*np.ones((Nobs,Nsamp)), b=bs, loc=logqo[:,None],
                          scale=sigma_logq[:,None], size=(Nobs,Nsamp))
    
    logqs_reweighted = []
    for i in range(Nobs):
        w = ss.norm.cdf(-logqo[i]/sigma_logq[i])/ss.norm.cdf(-logqs[i]/sigma_logq[i])
        logqs_reweighted.append(np.random.choice(logqs[i], size=Nsamp, p=w/np.sum(w), replace=True))
    logqs_reweighted = np.array(logqs_reweighted)

    return logqs_reweighted

def Theta_add_err(Theta, rho_obs, uncert, Nsamp):
    Nobs = Theta.shape[0]
    sigma_Theta = uncert['threshold_snr']/rho_obs*uncert['Theta']
    ao_T = -Theta/sigma_Theta
    bo_T = (1-Theta)/sigma_Theta
    Thetao = truncnorm.rvs(a=ao_T, b=bo_T, loc=Theta, scale=sigma_Theta, size=Nobs)

    as_T = []
    for i in range(Nobs):
        as_T.append(np.repeat(-Thetao[i]/sigma_Theta[i], Nsamp))
    as_T = np.array(as_T)
    
    bs_T = []
    for i in range(Nobs):
        bs_T.append(np.repeat((1-Thetao[i])/sigma_Theta[i], Nsamp))
    bs_T = np.array(bs_T)
    
    Thetas = truncnorm.rvs(a=as_T, b=bs_T, loc=Thetao[:,None],
                          scale=sigma_Theta[:,None], size=(Nobs,Nsamp))
    
    Thetas_reweighted = []
    for i in range(Nobs):
        w = (ss.norm.cdf((1-Thetao[i])/sigma_Theta[i]) - ss.norm.cdf(-Thetao[i]/sigma_Theta[i]))/(ss.norm.cdf((1-Thetas[i])/sigma_Theta[i]) - ss.norm.cdf(-Thetas[i]/sigma_Theta[i]))
        Thetas_reweighted.append(np.random.choice(Thetas[i], size=Nsamp, p=w/np.sum(w), replace=True))
    Thetas_reweighted = np.array(Thetas_reweighted)
    
    return Thetas_reweighted

def rhos_samples(rho_obs, Nsamp):
    Nobs = rho_obs.shape[0]
    rhos = np.random.normal(rho_obs[:,None], np.ones(Nobs)[:,None], size=(Nobs,Nsamp))
    return rhos

def dl_add_err(dl, Mczs, logqs, Thetas, rhos, uncert, Nsamp):
    Nobs = dl.shape[0]
    dfid = 1.
    ds = dfid*optimal_snr(Mczs, logqs, dfid)*Thetas/rhos
    
    return ds

def reweighted_samples(Mczs, logqs, Thetas, ds):
    Nobs = Mczs.shape[0]
    Nsamp = Mczs.shape[1]
    
    Mczs_reweighted = []
    logqs_reweighted = []
    Thetas_reweighted = []
    ds_reweighted = []
    
    for i in range(Nobs):
        dfid = 1.
        w = ds[i]**2/(Thetas[i]*optimal_snr(Mczs[i], logqs[i], dfid)*dfid)
        Mczs_reweighted.append(np.random.choice(Mczs[i], size=Nsamp, p=w/np.sum(w), replace=True))
        logqs_reweighted.append(np.random.choice(logqs[i], size=Nsamp, p=w/np.sum(w), replace=True))
        Thetas_reweighted.append(np.random.choice(Thetas[i], size=Nsamp, p=w/np.sum(w), replace=True))
        ds_reweighted.append(np.random.choice(ds[i], size=Nsamp, p=w/np.sum(w), replace=True))
    Mczs_reweighted = np.array(Mczs_reweighted)
    logqs_reweighted = np.array(logqs_reweighted)
    Thetas_reweighted = np.array(Thetas_reweighted)
    ds_reweighted = np.array(ds_reweighted)
    
    return Mczs_reweighted, logqs_reweighted, Thetas_reweighted, ds_reweighted

In [7]:
with h5py.File("mock_data.h5", "w") as file:
    for i in [1]:
        Nobs = 8000
        
        gm_samp, _ = gm.sample(Nobs)
        Mc, logq, z = gm_samp.T
        
        mask = z<0
        z[mask] = -z[mask]
        mask1 = logq>0
        logq[mask1] = -logq[mask1]
        
        Mcz = Mc*(1+z)
        dl = Planck18.luminosity_distance(z).to(u.Gpc).value
        
        Theta = np.random.beta(2, 4, Nobs)
        
        rho_obs = rho(optimal_snr(Mcz, logq, dl), Theta) + np.random.randn(Nobs)
        mask2 = rho_obs>rho_th
        rho_obs = rho_obs[mask2]
        Mcz = Mcz[mask2]
        logq = logq[mask2]
        dl = dl[mask2]
        Theta = Theta[mask2]
        Ndet = rho_obs.shape[0]
        print("Ndet=", Ndet, "among Nobs=", Nobs)
        
        Nsamp = 10000
        
        Mczs = mcz_add_err(Mcz, rho_obs, uncert, Nsamp)
        logqs = logq_add_err(logq, rho_obs, uncert, Nsamp)
        Thetas = Theta_add_err(Theta, rho_obs, uncert, Nsamp)
        
        rhos = rhos_samples(rho_obs, Nsamp)
        ds = dl_add_err(dl, Mczs, logqs, Thetas, rhos, uncert, Nsamp)
        
        #Mczs_reweighted, logqs_reweighted, Thetas_reweighted, ds_reweighted = reweighted_samples(Mczs, logqs, Thetas, ds)
        Mczs_reweighted, logqs_reweighted, Thetas_reweighted, ds_reweighted = Mczs, logqs, Thetas, ds
        
        file.create_dataset('Ndet'+str(i), data=Ndet)
        file.create_dataset('Mcz'+str(i), data=Mcz)
        file.create_dataset('logq'+str(i), data=logq)
        file.create_dataset('Theta'+str(i), data=Theta)
        file.create_dataset('dl'+str(i), data=dl)
        file.create_dataset('Mczs'+str(i), data=Mczs_reweighted)
        file.create_dataset('logqs'+str(i), data=logqs_reweighted)
        file.create_dataset('Thetas'+str(i), data=Thetas_reweighted)
        file.create_dataset('ds'+str(i), data=ds_reweighted)
file.close()

Ndet= 960 among Nobs= 8000


In [8]:
max(Mcz.flatten()), max(logq.flatten()), max(Theta.flatten()), max(dl.flatten())

(5.130497565784596,
 -1.2195204658480702e-05,
 0.9096135377275004,
 20.01604365977852)

In [9]:
min(Mcz.flatten()), min(logq.flatten()), min(Theta.flatten()), min(dl.flatten())

(1.0950128951986535,
 -0.7915646135459247,
 0.038664635638693726,
 0.00387182521530488)